In [ ]:
!pip install optimum transformers tiktoken einops cohere openai transformers_stream_generator accelerate gekko -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.5/402.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 28.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.3 MB/s

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/AutoGPTQ

/content/drive/MyDrive/AutoGPTQ


In [ ]:
# !git clone https://github.com/JustinLin610/AutoGPTQ.git

Cloning into 'AutoGPTQ'...
remote: Enumerating objects: 3879, done.
remote: Counting objects: 100% (3879/3879), done.
remote: Compressing objects: 100% (1350/1350), done.
remote: Total 3879 (delta 2556), reused 3752 (delta 2488), pack-reused 0
Receiving objects: 100% (3879/3879), 7.87 MiB | 9.08 MiB/s, done.
Resolving deltas: 100% (2556/2556), done.
Updating files: 100% (138/138), done.


In [ ]:
pip install -v .

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Processing /content/drive/MyDrive/AutoGPTQ
  Running command python setup.py egg_info
  Generating qigen kernels...
  conda_cuda_include_dir /usr/lib/python3/dist-packages/nvidia/cuda_runtime/include
  running egg_info
  creating /tmp/pip-pip-egg-info-opj2ts55/auto_gptq.egg-info
  writing /tmp/pip-pip-egg-info-opj2ts55/auto_gptq.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-opj2ts55/auto_gptq.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-opj2ts55/auto_gptq.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-opj2ts55/auto_gptq.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-opj2ts55/auto_gptq.egg-info/SOURCES.txt'
  /usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:502: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the sl

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
from sklearn.metrics import f1_score, accuracy_score
import torch
import numpy as np
import random
import gc
import os
import json
import pickle as pkl
import pandas as pd

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/21.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-VL:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL", device_map="cuda", trust_remote_code=True).eval()

config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

configuration_qwen.py:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-VL:
- configuration_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_qwen.py:   0%|          | 0.00/44.7k [00:00<?, ?B/s]

visual.py:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

...

pytorch_model-00008-of-00010.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00009-of-00010.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00010-of-00010.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/218 [00:00<?, ?B/s]

In [ ]:
def preprocess(df, name_to_value):
  id_to_prompt = {}
  id_to_label = {}

  df['prompt'] = df.apply(lambda row: f"What is the {row['attribute_names']} of this product: {row['texts']}? Answer with the option from the given choices directly: {name_to_value[row['attribute_names']]}.\nAnswer:", axis=1)


  for row in df.itertuples():
    id_to_prompt[row.id] = row.prompt
    id_to_label[row.id] = row.attribute_values

  return id_to_prompt, id_to_label

In [ ]:
def extract_answer(text):
    # Look for the pattern " Answer:" and extract the text immediately following it until the next quotation mark
    start_keyword = "Answer:"
    end_char = '"'

    # Find the starting position of the answer
    start_index = text.find(start_keyword)
    if start_index == -1:
        # If " Answer:" is not found, return an empty string or an error message
        return "Answer not found"

    # Adjust start_index to get the actual start of the answer text
    start_index += len(start_keyword) + 1

    # Find the end of the answer based on the next quotation mark
    end_index = text.find(end_char, start_index)
    if end_index == -1:
        # If the closing quotation mark is not found, return the substring from the start_index to the end of the string
        return text[start_index:]

    # Extract and return the answer
    return text[start_index:end_index]

In [ ]:
def inference(model, tokenizer, id_to_prompts, id_to_labels, img_dir):
  id_to_pred = {}
  preds = []
  labels = []

  for id in id_to_prompts:
    labels.append(id_to_labels[id])
    text = id_to_prompts[id]
    img_name = f'{id}.jpg'
    img_path = os.path.join(img_dir, img_name)

    if not os.path.exists(img_path):
      continue

    query = tokenizer.from_list_format([
    {'image': img_path},
    {'text': text},
      ])
    inputs = tokenizer(query, return_tensors='pt')
    inputs = inputs.to(model.device)
    pred = model.generate(**inputs)
    out = tokenizer.decode(pred.cpu()[0], skip_special_tokens=False)

    output = extract_answer(out)[:-13]


    preds.append(output)
    id_to_pred[id] = output

    torch.cuda.empty_cache()
    gc.collect()

  return id_to_pred, preds, labels

In [ ]:
def calculate_metrics(predictions, targets):
  binary_predictions = [1 if target.lower() in prediction.lower() else 0 for target, prediction in zip(targets, predictions)]
  binary_targets = [1] * len(targets)

  accuracy = sum(binary_predictions)
  micro_f1 = f1_score(binary_targets, binary_predictions, average='micro')
  return accuracy, micro_f1

In [ ]:
file_names = [
    'Clothing_annotated_final.tsv',
    'Jewlery_and_General_Apparel_annotated_final.tsv',
    'Footwear_annotated_final.tsv',
    'Food_annotated_final.tsv',
    'Home_annotated_final.tsv'
]

In [ ]:
img_dir = "../AVE/datasets/images"
for dataset in file_names:
  if "Food" in dataset:
    with open("../AVE/datasets/evaluation/options_Food.json", "r") as f:
      name_to_val = json.load(f)

  elif "Home" in dataset:
    with open("../AVE/datasets/evaluation/options_Home.json", "r") as f:
      name_to_val = json.load(f)

  else:
    with open("../AVE/datasets/evaluation/options_Clothing_Shoes_and_Jewelry.json", "r") as f:
      name_to_val = json.load(f)

  data = pd.read_csv(f'../AVE/datasets/evaluation/{dataset}', sep='\t')

  id_to_prompts, id_to_labels = preprocess(data, name_to_val)
  id_to_pred, preds, labels = inference(model, tokenizer, id_to_prompts, id_to_labels, img_dir)
  both_acc, both_micro_f1 = calculate_metrics(preds, labels)


  with open(f'../AVE/datasets/results/Qwen_Vl_7B_preds_both_{dataset[:-4]}.pkl', 'wb') as f:
      pkl.dump(id_to_pred, f)

  print(f'For {dataset[:-4]} multi modalities micro_f1 was {both_micro_f1}')

  torch.cuda.empty_cache()
  gc.collect()

# Prompt Testing Code

In [ ]:
def preprocess(df, name_to_value, custom_prompt):
    id_to_prompt = {}
    id_to_label = {}

    def apply_custom_prompt(row):
        return custom_prompt.format(
            attribute_names=row['attribute_names'],
            category=row['category'],
            texts=row['texts'],
            options=name_to_value[row['attribute_names']]
        )

    df['prompt'] = df.apply(apply_custom_prompt, axis=1)

    for row in df.itertuples():
        id_to_prompt[row.id] = row.prompt
        id_to_label[row.id] = row.attribute_values

    return id_to_prompt, id_to_label

In [ ]:
custom_prompts = [
    "Question: What is {attribute_names} of this product?\nContext: [Category] {category} {texts}.\nYou must only answer the question with exactly one of the following options {options}.\nAnswer:",
    "What is {attribute_names} of this product?[Category] {category} {texts}.Answer with the option from the given choices directly: {options}.\nAnswer:",
    "[Category] {category} {texts}. What is {attribute_names} of this product? Answer with the option from the given choices directly: {options}.\nAnswer:",
    "[Category] {category} {texts}. What is {attribute_names} of this product based on the given information and the given image? Answer with the option from the given choices directly: {options}.\nAnswer:",
    "[Category] {category} {texts}. Which one of {options} is the {attribute_names} of this product? Answer with the option from the given choices directly.\nAnswer:",
    "{texts}. What is the {attribute_names} of this product? Answer with the option from the given choices directly: {options}.\nAnswer:",
    "{texts}. Based on the description and the image, what is the {attribute_names} of this product? Answer with the option from the given choices directly: {options}.\nAnswer:",
    "What is the {attribute_names} of this product: {texts}? Answer with the option from the given choices directly: {options}.\nAnswer:"
]

In [ ]:
img_dir = "../AVE/datasets/image_final"
dataset = "Clothing_annotated_final.tsv"
with open("../AVE/datasets/evaluation/options_Clothing_Shoes_and_Jewelry.json", "r") as f:
    name_to_val = json.load(f)

for i, prompt in enumerate(custom_prompts):
  data = pd.read_csv(f'../AVE/datasets/evaluation/{dataset}', sep='\t')

  id_to_prompts, id_to_labels = preprocess(data, name_to_val, prompt)
  id_to_pred, preds, labels = inference(model, tokenizer, id_to_prompts, id_to_labels, img_dir)
  both_acc, both_micro_f1 = calculate_metrics(preds, labels)


  print(f'For prompt {i} the micro_f1 was {both_micro_f1}')

  torch.cuda.empty_cache()
  gc.collect()
